In [1]:
import h5py
import numpy as np
import scipy.constants as const
import sys
from astropy import units as un, constants as cons
import matplotlib.pyplot as plt
import matplotlib
import operator

h = 0.7
X = 0.7
mu = 0.62
R = const.R
kb = const.Boltzmann
Time_scale = 0.978

In [102]:
class Snapshot:
    def __init__(self,snapshotNum):
        self.num = snapshotNum
        f = h5py.File('snapshot_' + str(snapshotNum).zfill(3) + '.hdf5','r')
        self.header = f['Header']
        self.headerattrs = self.header.attrs
        self.gas = f['PartType0']
        self.dum1 = f['PartType2']
        self.dum2 = f['PartType3']
        #print("So far this is snapshot number: " + str(snapshotNum))
        if snapshotNum != 0:
            self.stars = f['PartType4']
            self.TSolarMass = np.sum(self.stars['Masses'])*(1e10/h) #in Msun
        self.BH = f['PartType5']
        self.Time = self.headerattrs['Time']*Time_scale #in Gyrs
        
    def rhos(self):  # in g/m^3
        return ((un.Msun / un.kpc ** 3).to('g/m**3') *
                np.array(list(self.gas['Density'])) * 1e10)

    def nHs(self):  # in m^-3
        return X * self.rhos() / cons.m_p.to('g').value
    
    def Temperature(self):  # in K
        internalEnergy = self.gas['InternalEnergy']  # energy per unit mass
        return (un.km ** 2 / un.s ** 2 * cons.m_p / cons.k_B).to('K').value * (2. / 3 * mu) * internalEnergy

#     def totalSolarMass(self):
#         return np.sum(self.stars['Masses'])
    
    def diffSFR(self):
        try:
            nxtsnp = Snapshot(self.num + 1)
        except FileNotFoundError:
                print("The SFR is calculated using the total solar mass of the next snapshot, there for if try to determine the SFR of the last snapshot you cant. Will return 0")
                return 0
        if (self.num ==0):
            return Snapshot(1).TSolarMass
        return nxtsnp.TSolarMass - self.TSolarMass
    
    def totalSFR(self):
        totalSFR = 0
        for i in range(len(self.gas['StarFormationRate'])):
            if self.gas['StarFormationRate'][i] > 0:
                totalSFR += self.gas['StarFormationRate'][i]
        return totalSFR
    
    
    def sfr(self):
        sfr = {}
        for i in range(len(self.gas['StarFormationRate'])):
            if self.gas['StarFormationRate'][i] > 0:
                sfr[str(self.gas['ParticleIDs'][i])] = self.gas['StarFormationRate'][i]
                print(i)   
        return sfr
                
    def sfr2(self):
        for id in self.gas["ParticleIDs"]:
            if self.gas['StarFormationRate'][id] > 0:
                print('Particle number: ' + str(id))
                print("Particle SFR: "+ str(self.gas['StarFormationRate'][id])+ ' MSun per year')
    
    def radius(self): # in kpc
        return np.array(self.gas['BH_Dist'])*0.7
    
    
    def gasPressure(self): #in J/m^3=Pa
        n = self.nHs()
        P = 2.3 * n * self.Temperature() * cons.k_B.value
        return P

    
    def fill_zeros_smoothly(self,arr):
        nonzero_indices = np.nonzero(arr)[0]
        zero_indices = np.where(arr == 0)[0]
        for zero_index in zero_indices:
            left_nonzero_index = np.searchsorted(nonzero_indices, zero_index) - 1
            if left_nonzero_index == -1:
                right_nonzero_index = 1
            elif zero_index == arr.shape[0] - 1:
                left_nonzero_index = nonzero_indices.shape[0] - 2
                right_nonzero_index = left_nonzero_index + 1
            else:
                right_nonzero_index = left_nonzero_index + 1
            left_nonzero_value = arr[nonzero_indices[left_nonzero_index]]
            right_nonzero_value = arr[nonzero_indices[right_nonzero_index]]
            arr[zero_index] = (left_nonzero_value + right_nonzero_value) / 2
        return arr
    
    def fixPressures(self,aveP,DP):
        #print(self.num)
        N = len(aveP)
        for j in range(N):
            if aveP[j] == 0:
                i = 0
                try: 
                    while aveP[j+i] == 0:
                        i += 1
                except IndexError:
                    aveP[j] = aveP[j-1]
                if j-i<0:
                    aveP[j] = np.mean(aveP[:j+i+1])
                else: 
                    aveP[j] = np.mean(aveP[j-i:j+i+1])
            if DP[j] == 0:
                i = 0
                try: 
                    while DP[j+i] == 0:
                        i += 1
                except IndexError:                        
                    DP[j] = DP[j-1]
                if j-i<0:
                    DP[j] = np.mean(DP[:j+i+1])
                else: 
                    DP[j] = np.mean(DP[j-i:j+i+1])
            
    
    def avePressure(self):
        r0 = 0.1 #in kpc
        rN = 1e3 #Mpc in kpc
        P = self.gasPressure()
        M = self.gas['Masses']
        radiuses = self.radius()
        r = []
        aveP = []
        DP = []
        while r0 < rN:
            dr = 0.1*r0
            indecies = np.where((radiuses>r0) & (radiuses<(r0+dr)))
            getter = operator.itemgetter(*list(indecies))
            if indecies[0].size == 0:
                aveP.append(0)
                DP.append(0)
            else: 
                aveP.append(np.average(getter(P),weights=getter(M)))
                DP.append(np.sqrt(np.mean((getter(P) - aveP[-1])**2)))
            r.append(r0)
            r0 += dr
        self.fixPressures(aveP,DP)
#         aveP = self.fill_zeros_smoothly(aveP)
#         DP = self.fill_zeros_smoothly(DP)
        print("Finished with snapshot number: " + str(self.num))
        return (aveP,DP)
    
    def avePressure2(self):
        r0 = 0.1 #in kpc
        rN = 1e3 #Mpc in kpc
        P = self.gasPressure()
        M = self.gas['Masses']
        radiuses = self.radius()
        r = []
        aveP = []
        DP = []
        while r0 < rN:
            dr = 0.1*r0
            indecies = np.where((radiuses>r0) & (radiuses<(r0+dr)))
            getter = operator.itemgetter(*list(indecies))
#             print(indecies[0].size)
            if indecies[0].size == 0:
#                 print("wut")
                aveP.append(0)
                DP.append(0)
            else: 
                try:
                    aveP.append(np.average(getter(P),weights=getter(M)))
                    DP.append(np.sqrt(np.mean((getter(P) - aveP[-1])**2)))
#                     if self.num == 75:
#                         print("P are: " + str(getter(P)))
#                         print("aveP is: " + str(aveP[-1]))
#                         print("DP is: " + str(DP[-1]))
                except ZeroDivisionError:
#                     print("snapshot number: " + str(self.num))
#                     print(indecies[0])
#                     print(getter(M))
#                     print("The sum is: " + str(np.sum(getter(M))))
            r.append(r0)
            r0 += dr
        self.fixPressures(aveP,DP)
#         aveP = self.fill_zeros_smoothly(aveP)
#         DP = self.fill_zeros_smoothly(DP)
        print("Finished with snapshot number: " + str(self.num))
        return (aveP,DP)